<a href="https://colab.research.google.com/github/cristinarainich/Sternberg_analysis/blob/main/Environmental.%20Dataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

Updated 16/05 15:45

In [ ]:
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [139]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental')

In [ ]:
!ls /content/gdrive/MyDrive/CoganLab/Environmental

 D29_Block_1_TrialData.mat	  eeg_envir.csv
 D64_Block_1_TrialData.mat	  eeg_envir_processed.csv
 D65_Block_1_TrialData_1.mat	  Environmental_Subject.mat
 D65_Block_1_TrialData.mat	  healthy
'D71_Block_1_TrialData (1).mat'   ieeg_envir.csv
'D73_Block_1_TrialData (2).mat'   ieeg_envir_processed.csv
'D77_Block_1_TrialData (1).mat'   stimuli


In [140]:
# Patients
ieeg = scipy.io.loadmat(
    working_path.joinpath('Environmental_Subject.mat')
    )

subj1 = scipy.io.loadmat(
    working_path.joinpath('D65_Block_1_TrialData.mat')
    )

subj65 = scipy.io.loadmat(
    working_path.joinpath('D65_Block_1_TrialData_1.mat')
    )

subj64 = scipy.io.loadmat(
    working_path.joinpath('D64_Block_1_TrialData.mat')
)
subj73 = scipy.io.loadmat(
    working_path.joinpath('D73_Block_1_TrialData (2).mat')
    )
subj77 = scipy.io.loadmat(
    working_path.joinpath('D77_Block_1_TrialData (1).mat')
    )

In [ ]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental/healthy')

In [ ]:
# healthy
files = [] 
begin = 'E'
end = '_Block_1_TrialData.mat'
files.append(begin + '32' + end)
for i in range(34, 47):
  files.append(begin + str(i) + end)

In [ ]:
files[0]

'E32_Block_1_TrialData.mat'

In [ ]:
eeg = []
for i in files:
  subset = scipy.io.loadmat(
      working_path.joinpath(i)
  )
  eeg.append(subset)

#Working with stimuli

In [ ]:
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental/stimuli')

In [ ]:
!ls /content/gdrive/MyDrive/CoganLab/Environmental/stimuli

imaginability.txt	 proposed_nw.mat	 proposed_w3.mat  untrn_w.mat
proposed_nw3_global.mat  proposed_w2.mat	 proposed_w.mat
proposed_nw3.mat	 proposed_w3_global.mat  untrn_nw.mat


In [ ]:
stimuli_words = scipy.io.loadmat(
    working_path.joinpath('proposed_w3.mat')
)

In [ ]:
columns = stimuli_words['proposed_w'].dtype.names
stim_words = pd.DataFrame(columns=columns)
for i in range(0, stimuli_words['proposed_w'].shape[1]):
  values = []
  subset = stimuli_words['proposed_w'][0][i]
  for name in columns:
    values.append(subset[name])
  stim_words.loc[len(stim_words.index)] = values

In [ ]:
stimuli = []
for i in stim_words.Word.values:
  stimuli.append(i[0])

freq = []
for i in stim_words.SFreq.values:
  freq.append(i[0][0])

imagine = []
for i in stim_words.imaginability.values:
  imagine.append(i[0][0])

category = []
for i in range(0, 24):
  category.append('word')

In [ ]:
columns = ['Category', 'Stimulus', 'Frequency', 'Imaginability']
words = pd.DataFrame(columns=columns)
words['Category'] = category
words['Stimulus'] = stimuli
words['Frequency'] = freq
words['Imaginability'] = imagine

In [ ]:
stimuli_nonwords = scipy.io.loadmat(
    working_path.joinpath('proposed_nw3.mat')
)

In [ ]:
columns = stimuli_nonwords['proposed_nw'].dtype.names
stim_nonwords = pd.DataFrame(columns=columns)
for i in range(0, stimuli_nonwords['proposed_nw'].shape[1]):
  values = []
  subset = stimuli_nonwords['proposed_nw'][0][i]
  for name in columns:
    values.append(subset[name])
  stim_nonwords.loc[len(stim_nonwords.index)] = values

In [ ]:
stimuli = []
for i in stim_nonwords.Word.values:
  stimuli.append(i[0])

freq = []
for i in stim_nonwords.SFreq.values:
  freq.append(i[0][0])

category = []
for i in range(0, 24):
  category.append('nonword')

In [ ]:
columns = ['Category', 'Stimulus', 'Frequency', 'Imaginability']
nonwords = pd.DataFrame(columns=columns)
nonwords['Category'] = category
nonwords['Stimulus'] = stimuli
nonwords['Frequency'] = freq
nonwords['Imaginability'] = np.nan

In [ ]:
all_stim = pd.concat([words, nonwords])

# iEEG dataset

In [141]:
data_ieeg = ieeg['Subject']
print('Number of subjects in iEEG dataset is', data_ieeg.shape[1])

Number of subjects in iEEG dataset is 19


In [142]:
subjects_ieeg = [] # subject numbers
for i in data_ieeg['Name'][0]:
  subjects_ieeg.append(i[0])
print(subjects_ieeg)

['D27', 'D28', 'D30', 'D31', 'D32', 'D35', 'D37', 'D38', 'D39', 'D41', 'D42', 'D47', 'D53', 'D54', 'D55', 'D57', 'D63', 'D69', 'D71']


In [143]:
columns = []
for i in range(0, data_ieeg.shape[1]):
  actual = data_ieeg[0][i]['trialInfo'][0][0].dtype.names
  for name in actual:
    if name not in columns:
      columns.append(name)

In [144]:
def get_to_df(data):
  df = pd.DataFrame(columns=columns)
  df['Trial'] = np.nan
  df['Subject'] = np.nan

  for i in range(0, data.shape[1]):
    # for each subject
    subset = data[0][i]
    subject = subset['Name'][0]
    print(subset['Task'])
    print(subject)
    for trial in range(0, subset['trialInfo'].shape[1]):
      # for each trial
      sub = subset['trialInfo'][0][trial]
      dtype = sub.dtype.names
      values = []
      for name in columns:
        if name in dtype:
          values.append(sub[name])
        else:
          values.append(np.nan)
      values.append(str(trial)) # adding a trial number
      values.append(subject)
      df.loc[len(df.index)] = values
  return df

In [145]:
ieeg = get_to_df(data_ieeg)

['Environmental_Sternberg']
D27
['Environmental_Sternberg']
D28
['Environmental_Sternberg']
D30
['Environmental_Sternberg']
D31
['Environmental_Sternberg']
D32
['Environmental_Sternberg']
D35
['Environmental_Sternberg']
D37
['Environmental_Sternberg']
D38
['Environmental_Sternberg']
D39
['Environmental_Sternberg']
D41
['Environmental_Sternberg']
D42
['Environmental_Sternberg']
D47
['Environmental_Sternberg']
D53
['Environmental_Sternberg']
D54
['Environmental_Sternberg']
D55
['Environmental_Sternberg']
D57
['Environmental_Sternberg']
D63
['Environmental_Sternberg']
D69
[]
D71


In [146]:
# adding subjects D64
df_64 = pd.DataFrame(columns=columns)
df_64['Trial'] = np.nan
df_64['Subject'] = np.nan
for trial in range(0, subj64['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj64['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D64')
  df_64.loc[len(df_64.index)] = values

In [147]:
# adding subjects D65
df_65 = pd.DataFrame(columns=columns)
df_65['Trial'] = np.nan
df_65['Subject'] = np.nan
for trial in range(0, subj1['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj1['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D65')
  df_65.loc[len(df_65.index)] = values
  i = 96
for trial in range(0, subj65['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj65['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(i)) # adding a trial number
  values.append('D65')
  i += 1
  df_65.loc[len(df_65.index)] = values

In [148]:
ieeg = pd.concat([ieeg, df_64])
ieeg = pd.concat([ieeg, df_65])

In [149]:
# adding subject 73
df_73 = pd.DataFrame(columns=columns)
df_73['Trial'] = np.nan
df_73['Subject'] = np.nan
for trial in range(0, subj73['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj73['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D73')
  df_73.loc[len(df_73.index)] = values

In [150]:
# adding subject 77
df_77 = pd.DataFrame(columns=columns)
df_77['Trial'] = np.nan
df_77['Subject'] = np.nan
for trial in range(0, subj77['trialInfo'].shape[1]):
  # for each trial
  values = []
  subset = subj77['trialInfo'][0][trial]
  dtype = subset.dtype.names
  for name in columns:
    if name in dtype:
      values.append(subset[name])
    else:
      values.append(np.nan)
  values.append(str(trial)) # adding a trial number
  values.append('D77')
  df_77.loc[len(df_77.index)] = values

In [151]:
ieeg = pd.concat([ieeg, df_73])
ieeg = pd.concat([ieeg, df_77])

In [152]:
ieeg.groupby(['Subject'])['Trial'].count()

Subject
D27    160
D28    120
D30    168
D31    168
D32    168
D35    168
D37    168
D38    160
D39    168
D41    168
D42    168
D47    168
D53    168
D54    168
D55    168
D57    168
D63    168
D64    168
D65    168
D69    168
D71    168
D73    168
D77    168
Name: Trial, dtype: int64

In [153]:
# saving ieeg dataset
working_path = Path('/content/gdrive/MyDrive/CoganLab/Environmental')
ieeg.to_csv(
    Path(working_path).joinpath('ieeg_envir.csv')
)

# Preprocessing of iEEG dataset

In [ ]:
ieeg = pd.read_csv(
    working_path.joinpath('ieeg_envir.csv')
    )

In [ ]:
ieeg_alter = pd.read_csv(
    working_path.joinpath('ieeg_envir_processed.csv')
    )

In [ ]:
ieeg_alter = ieeg_alter.drop('Unnamed: 0', axis=1)

In [154]:
ieeg.Subject.unique()

array(['D27', 'D28', 'D30', 'D31', 'D32', 'D35', 'D37', 'D38', 'D39',
       'D41', 'D42', 'D47', 'D53', 'D54', 'D55', 'D57', 'D63', 'D69',
       'D71', 'D64', 'D65', 'D73', 'D77'], dtype=object)

In [155]:
ieeg_ = ieeg[['block', 'ProbeType', 'StimlusCategory',
       'ReactionTime', 'Resp', 'RespCorrect', 'Omission',
       'stimulusSounds_idx', 'stimulusSounds_name', 'ProbeCategory', 'probeSound_idx',
       'probeSound_name', 'StimulusCategory', 'Trial',
       'Subject']]

In [156]:
# looks like D27 performed Neighbourhood here
# let's exclude them
ieeg_ = ieeg_[ieeg_['Subject'] != 'D27']

In [157]:
ieeg_ = ieeg_.drop(['Omission', 'ProbeType'], axis=1)

In [159]:
# working on the columns to make them more convinient to use
block = ieeg_.block.values
new_block = []
for i in block:
  while type(i) is np.ndarray:
    i = i[0]
  new_block.append(i)

In [161]:
rt = ieeg_.ReactionTime.values
new_rt = []
for i in rt:
  while type(i) is np.ndarray:
    i = i[0]
  new_rt.append(i)

In [163]:
acc = ieeg_.RespCorrect.values
new_acc = []
for i in acc:
  while type(i) is np.ndarray:
    i = i[0]
  new_acc.append(i)

In [165]:
resp = ieeg_.Resp.values
new_resp = []
for i in resp:
  while type(i) is np.ndarray:
    i = i[0]
  new_resp.append(i)

In [167]:
probe = ieeg_.ProbeCategory.values
new_probe = []
for i in probe:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe.append(i)

In [169]:
probe_idx = ieeg_.probeSound_idx.values
new_probe_idx = []
for i in probe_idx:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe_idx.append(i)

In [173]:
stim_idx = ieeg_.stimulusSounds_idx.values
new_stim_idx = []
for i in stim_idx:
  new_i = i[0][0].tolist()
  element = []
  for j in new_i:
    element.append(j[0])
  new_stim_idx.append(element)

In [185]:
probe_name = ieeg_.probeSound_name.values
new_probe_name = []
for i in probe_name:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe_name.append(i)

In [203]:
stim_name = ieeg_.stimulusSounds_name.values
new_stim_name = []
for i in stim_name:
  new_i = i[0][0].tolist()
  element = []
  for j in new_i:
    element.append(j[0][0][0])
  new_stim_name.append(element)

In [225]:
stim1[0]

array([[array(['nonwords'], dtype='<U8')]], dtype=object)

In [224]:
stim1 = ieeg_.StimlusCategory.values
stim2 = ieeg_.StimulusCategory.values
new_stim = []
for i in range(0, ieeg_.shape[0]):
  if type(stim1[i]) is np.ndarray:
    while type(stim1[i]) is np.ndarray:
      element = stim1[i][0]
    new_stim.append(element)
  elif type(stim2[i]) is np.ndarray:
    while type(stim2[i]) is np.ndarray:
      element = stim2[i][0]
    new_stim.append(element)
  else:
    print(stim1[i], stim2[i])
    break

KeyboardInterrupt: ignored

In [220]:
new_stim

[]

In [219]:
ieeg_[ieeg_['Subject'] == 'D32']

,block,StimlusCategory,ReactionTime,Resp,RespCorrect,stimulusSounds_idx,stimulusSounds_name,ProbeCategory,probeSound_idx,probeSound_name,StimulusCategory,Trial,Subject
616,[[[[1]]]],NaN,[[[[1.22790125]]]],[[[No]]],[[[[1]]]],"[[[[68], [69], [64], [59], [54]]]]","[[[[array(['thowster.wav'], dtype='<U12')], [a...",[[[[0]]]],[[[[23]]]],[[[yehbahl.wav]]],[[[nonwords]]],0,D32
617,[[[[1]]]],NaN,[[[[0.95329369]]]],[[[Yes]]],[[[[0]]]],"[[[[10], [6], [4], [2], [16], [7], [22], [14],...","[[[[array(['duck_quack.wav'], dtype='<U14')], ...",[[[[0]]]],[[[[8]]]],[[[dog_panting.wav]]],[[[environment]]],1,D32
618,[[[[1]]]],NaN,[[[[1.31482907]]]],[[[No]]],[[[[1]]]],"[[[[46], [36], [42], [34], [29]]]]","[[[[array(['tunic.wav'], dtype='<U9')], [array...",[[[[0]]]],[[[[2]]]],[[[chalice.wav]]],[[[words]]],2,D32
619,[[[[1]]]],NaN,[[[[0.89415912]]]],[[[Yes]]],[[[[1]]]],"[[[[36], [33], [28], [27], [46]]]]","[[[[array(['madam.wav'], dtype='<U9')], [array...",[[[[1]]]],[[[[3]]]],[[[chapter.wav]]],[[[words]]],3,D32
620,[[[[1]]]],NaN,[[[[0.86633712]]]],[[[No]]],[[[[0]]]],"[[[[71], [51], [55], [61], [59]]]]","[[[[array(['yehbahl.wav'], dtype='<U11')], [ar...",[[[[1]]]],[[[[7]]]],[[[haasahp.wav]]],[[[nonwords]]],4,D32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,[[[[7]]]],NaN,[[[[1.09748251]]]],[[[Yes]]],[[[[1]]]],"[[[[6], [13], [7], [4], [8]]]]","[[[[array(['crowd_cheering.wav'], dtype='<U18'...",[[[[1]]]],[[[[13]]]],[[[gargling.wav]]],[[[environment]]],163,D32
780,[[[[7]]]],NaN,[[[[1.47197723]]]],[[[No]]],[[[[1]]]],"[[[[70], [56], [66], [69], [49], [67], [54], [...","[[[[array(['wiykaan.wav'], dtype='<U11')], [ar...",[[[[0]]]],[[[[3]]]],[[[drahjer.wav]]],[[[nonwords]]],164,D32
781,[[[[7]]]],NaN,[[[[1.17343654]]]],[[[Yes]]],[[[[1]]]],"[[[[32], [38], [45], [27], [39], [29], [47], [...","[[[[array(['drama.wav'], dtype='<U9')], [array...",[[[[1]]]],[[[[11]]]],[[[kitchen.wav]]],[[[words]]],165,D32
782,[[[[7]]]],NaN,[[[[1.60527002]]]],[[[Yes]]],[[[[1]]]],"[[[[50], [57], [66], [63], [51]]]]","[[[[array(['chuwpahl.wav'], dtype='<U12')], [a...",[[[[1]]]],[[[[18]]]],[[[tehlper.wav]]],[[[nonwords]]],166,D32


In [ ]:
ieeg_['block'] =


In [ ]:
ieeg_alter.head()

In [205]:
ieeg_.head()

,block,StimlusCategory,ReactionTime,Resp,RespCorrect,stimulusSounds_idx,stimulusSounds_name,ProbeCategory,probeSound_idx,probeSound_name,StimulusCategory,Trial,Subject
160,[[[[1]]]],[[[nonwords]]],[[[[0.99043604]]]],[[[No]]],[[[[1]]]],"[[[[3], [7], [15]]]]","[[[[(array(['camera_snapping_photos.wav'], dty...",[[[[0]]]],[[[[14]]]],[[[telephone_dialing.wav]]],NaN,0,D28
161,[[[[1]]]],[[[words]]],[[[[1.30217997]]]],[[[Yes]]],[[[[1]]]],"[[[[16], [7], [14], [13], [9], [11], [6]]]]","[[[[(array(['typing.wav'], dtype='<U10'), arra...",[[[[1]]]],[[[[11]]]],[[[hair_dryer.wav]]],NaN,1,D28
162,[[[[1]]]],[[[environment]]],[[[[1.43263021]]]],[[[Yes]]],[[[[1]]]],"[[[[17], [15], [13], [5], [9], [10], [7], [6],...","[[[[(array(['writing_on_paper.wav'], dtype='<U...",[[[[1]]]],[[[[7]]]],[[[cutting_with_scissors.wav]]],NaN,2,D28
163,[[[[1]]]],[[[environment]]],[[[[1.16073421]]]],[[[Yes]]],[[[[1]]]],"[[[[17], [1], [13]]]]","[[[[(array(['writing_on_paper.wav'], dtype='<U...",[[[[1]]]],[[[[13]]]],[[[stream.wav]]],NaN,3,D28
164,[[[[1]]]],[[[environment]]],[[[[0.84571169]]]],[[[Yes]]],[[[[1]]]],"[[[[16], [7], [10], [15], [12], [3], [2]]]]","[[[[(array(['typing.wav'], dtype='<U10'), arra...",[[[[1]]]],[[[[3]]]],[[[camera_snapping_photos.wav]]],NaN,4,D28


In [ ]:
ieeg_.StimulusCategory.values[180:210]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan], dtype=object)

# EEG dataset

In [ ]:
subjects = []
for i in files:
  subjects.append(i[:3])

In [ ]:
columns = []
for subject in eeg:
  # for each subject
  actual = subject['trialInfo'][0][0].dtype.names
  for name in actual:
    if name not in columns:
      columns.append(name)

In [ ]:
df_ = pd.DataFrame(columns=columns)
df_['Trial'] = np.nan
df_['Subject'] = np.nan
for subject in eeg:
  for trial in range(0, subject['trialInfo'].shape[1]):
    # for each trial
    values = []
    subset = subject['trialInfo'][0][trial]
    dtype = subset.dtype.names
    for name in columns:
      if name in dtype:
        values.append(subset[name])
      else:
        print('check')
        values.append(np.nan)
    values.append(str(trial))
    values.append(subjects[eeg.index(subject)])
    df_.loc[len(df_.index)] = values

In [ ]:
eeg = df_.copy()

In [ ]:
eeg.groupby(['Subject'])['Trial'].count()

Subject
E32    216
E34    216
E35    216
E36    216
E37    216
E38    216
E39    216
E40    184
E41    216
E42    216
E43    216
E44    216
E45    216
E46    216
Name: Trial, dtype: int64

In [ ]:
# saving eeg file 
eeg.to_csv(
    Path(working_path).joinpath('eeg_envir.csv')
)

#Preprocessing of EEG dataset

In [ ]:
eeg = pd.read_csv(
    working_path.joinpath('eeg_envir.csv')
    )

In [ ]:
eeg.Subject.unique()

array(['E32', 'E34', 'E35', 'E36', 'E37', 'E38', 'E39', 'E40', 'E41',
       'E42', 'E43', 'E44', 'E45', 'E46'], dtype=object)

In [ ]:
len(eeg.Subject.unique())

14

In [ ]:
eeg_ = eeg[[
    'block', 'StimulusCategory', 'ReactionTime',
       'Resp', 'RespCorrect', 'stimulusSounds_idx',
       'ProbeCategory', 'probeSound_idx',
       'Trial', 'Subject', 'stimulusSounds_name', 'probeSound_name'
]]

Getting rid of additional lists containing lists

In [ ]:
block = eeg_.block.values
new_block = []
for i in block:
  while type(i) is np.ndarray:
    i = i[0]
  new_block.append(i)

In [ ]:
stim_cat = eeg_.StimulusCategory.values
new_stim = []
for i in stim_cat:
  if 'environment' in i:
    new_stim.append('environment')
  elif 'nonwords' in i:
    new_stim.append('nonwords')
  elif 'words' in i:
    new_stim.append('words')

In [ ]:
rt = eeg_.ReactionTime.values
new_rt = []
for i in rt:
  while type(i) is np.ndarray:
    i = i[0]
  new_rt.append(i)

In [ ]:
acc = eeg_.RespCorrect.values
new_acc = []
for i in acc:
  while type(i) is np.ndarray:
    i = i[0]
  new_acc.append(i)

In [ ]:
resp = eeg_.Resp.values
new_resp = []
for i in resp:
  while type(i) is np.ndarray:
    i = i[0]
  new_resp.append(i)

In [ ]:
probe = eeg_.ProbeCategory.values
new_probe = []
for i in probe:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe.append(i)

In [ ]:
probe_idx = eeg_.probeSound_idx.values
new_probe_idx = []
for i in probe_idx:
  while type(i) is np.ndarray:
    i = i[0]
  new_probe_idx.append(i)

In [ ]:
stim_idx = eeg_.stimulusSounds_idx.values
new_stim_idx = []
for i in stim_idx:
  new_i = i[0][0].tolist()
  element = []
  for j in new_i:
    element.append(j[0])
  new_stim_idx.append(element)

In [ ]:
eeg_['block'] = new_block
eeg_['StimulusCategory'] = new_stim
eeg_['ReactionTime'] = new_rt
eeg_['RespCorrect'] = new_acc
eeg_['Resp'] = new_resp
eeg_['ProbeCategory'] = new_probe
eeg_['probeSound_idx'] = new_probe_idx
eeg_['stimulusSounds_idx'] = new_stim_idx

<ipython-input-58-ea534a877bb7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['block'] = new_block
<ipython-input-58-ea534a877bb7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['StimulusCategory'] = new_stim
<ipython-input-58-ea534a877bb7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [ ]:
length = []
for i in eeg_.stimulusSounds_idx.values:
  if len(i) not in length:
    length.append(len(i))
print(length)

[3, 7, 9, 5]


In [ ]:
length = []
for i in eeg_.stimulusSounds_idx.values:
  length.append(len(i))
eeg_['Length'] = length

<ipython-input-60-d50296b58efb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Length'] = length


In [ ]:
words = []
nonwords = []
environment = []
for i in eeg_.StimulusCategory.values:
  if i == 'nonwords':
    words.append(0)
    nonwords.append(1)
    environment.append(0)
  elif i == 'words':
    words.append(1)
    nonwords.append(0)
    environment.append(0)
  elif i == 'environment':
    words.append(0)
    nonwords.append(0)
    environment.append(1)

In [ ]:
eeg_['Words'] = words
eeg_['Nonwords'] = nonwords
eeg_['Environment'] = environment

<ipython-input-62-3c31c9b484ea>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Words'] = words
<ipython-input-62-3c31c9b484ea>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['Nonwords'] = nonwords
<ipython-input-62-3c31c9b484ea>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

In [ ]:
probe_names = eeg_.probeSound_name.values
new_probe_names = []
for name in probe_names:
  name = name[0][0][0]
  new_probe_names.append(name)

In [ ]:
stimulus_names = eeg_.stimulusSounds_name.values
new_stim_names = []
for name in stimulus_names:
  name = name[0][0]
  new_name = []
  for j in name:
    new_name.append(j[0][0])
  new_stim_names.append(new_name)

In [ ]:
new_new_stim_names = []
for element in new_stim_names:
  new_element = []
  for name in element:
    new_element.append(name[:-4])
  new_new_stim_names.append(new_element)

In [ ]:
new_new_probe_names = []
for i in new_probe_names:
  new_new_probe_names.append(i[:-4])

In [ ]:
eeg_['stimulusSounds_name'] = new_new_stim_names
eeg_['probeSound_name'] = new_new_probe_names

<ipython-input-67-a747f04adf7e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['stimulusSounds_name'] = new_new_stim_names
<ipython-input-67-a747f04adf7e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['probeSound_name'] = new_new_probe_names


**Adding frequency and imaginabilty scores**

In [ ]:
category = eeg_.StimulusCategory.values.tolist()
probe = eeg_.probeSound_name.values
stimuli = eeg_.stimulusSounds_name.values
standard = all_stim[all_stim['Category'] == 'word']['Stimulus'].values.tolist()
standard_freq = all_stim[all_stim['Category'] == 'word']['Frequency'].values.tolist()
standard_imag = all_stim[all_stim['Category'] == 'word']['Imaginability'].values.tolist()

stim_freq = []
stim_imag = []
probe_freq = []
probe_imag = []

for i in range(0, eeg_.shape[0]):
  if category[i] == 'words':
    for name in standard:
      if probe[i] == name:
        freq = standard_freq[standard.index(name)]
        imag = standard_imag[standard.index(name)]
        probe_freq.append(freq)
        probe_imag.append(imag)

    values_freq = []
    values_imag = []      
    for lis in stimuli[i]:
      for name in standard:
        if lis == name:
          freq = standard_freq[standard.index(name)]
          imag = standard_imag[standard.index(name)]
          values_freq.append(freq)
          values_imag.append(imag)
    stim_freq.append(values_freq)
    stim_imag.append(values_imag)
  
  else:
    stim_freq.append(np.nan)
    stim_imag.append(np.nan)
    probe_freq.append(np.nan)
    probe_imag.append(np.nan)

In [ ]:
print(eeg_.shape)
print(len(stim_freq))
print(len(stim_imag))
print(len(probe_freq))
print(len(probe_imag))

(2992, 16)
2992
2992
2992
2992


In [ ]:
eeg_['ProbeFreq'] = probe_freq
eeg_['ProbeImag'] = probe_imag
eeg_['StimFreq'] = stim_freq
eeg_['StimImag'] = stim_imag

<ipython-input-70-c1691c97d5de>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['ProbeFreq'] = probe_freq
<ipython-input-70-c1691c97d5de>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eeg_['ProbeImag'] = probe_imag
<ipython-input-70-c1691c97d5de>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [ ]:
# saving the file
eeg_.to_csv(
    Path(working_path).joinpath('eeg_envir_processed.csv')
)